In [1]:
from sqlalchemy import create_engine


engine = create_engine('mysql+pyodbc://telecom')
%load_ext sql
%sql engine
%config SqlMagic.autopandas = True

Found pyproject.toml from '/Users/jacky/github/telecom'

In [2]:
%sql SHOW tables

Running query in 'mysql+pyodbc://telecom'

10 rows affected.

,Tables_in_telecom
0,cinfo_xy
1,sample_sight_201308
2,sample_user_201308
3,serv_acct_item_0838_201308
4,serv_acct_item_0838_201406
5,tb_asz_cdma_0838_201308
6,tb_asz_cdma_0838_201406
7,tower
8,tower_center
9,tower_hot_201308


# creation of the table *tower_hot_201308*

```sql
CREATE TABLE sight_xy_201308 (
	SELECT 
        A.SERV_ID, A.ACC_NBR, A.MONTH_NO, A.LAC, A.CELL_ID, B.LON, B.LAT, B.X, B.Y
	    SUBSTRING_INDEX(A.START_TIME,' ',1) AS DATE,
        DAYOFWEEK(SUBSTRING_INDEX(A.START_TIME,' ',1)) AS WEEK_DAY,
	    CONVERT(SUBSTRING_INDEX(SUBSTRING_INDEX(A.START_TIME,' ',-1),':',1),UNSIGNED) AS HOUR_NO,
	FROM serv_acct_item_0838_201308 A JOIN tower B
	ON A.CELL_ID=B.CELL_16
	WHERE 
        A.LAC!='-1' AND 
        A.LAC!='0' AND 
        A.START_TIME LIKE '____/%/% %:__:__'
);
CREATE INDEX xy_sight_201308 ON sight_xy_201308(LON, LAT);
CREATE INDEX weekhour_xy_201308 ON sight_xy_201308(WEEK_DAY, HOUR_NO);
```

- The table *sight_xy_201308* is created by jointing calling records with the tower informatin on *cell*
- We can also observe that the calling records' *CELL_ID* share similarity with tower information's *CELL_16*
- Moreover, outliers include -1, 0 in *LAC*
- More information of *LAC(location area code)* can be found [here](https://www.simbase.com/iot-glossary-dictionary/location-area-code).

### experiments - SUBSTRING_INDEX, DAYOFWEEK
- join statement with limit shuld use 
- day of week start at Sunday(1), So 5 means Thursday
- SUBSTRING_INDEX(string, delimiter, number)
    - An integer indicating the number of occurrences of delim.

In [23]:
%%sql


(
    SELECT
        'original' AS format,
         START_TIME AS time
    FROM serv_acct_item_0838_201308
    LIMIT 1
)
UNION
(
    SELECT
        'get_date' AS format,
         SUBSTRING_INDEX(START_TIME,' ',1) AS time
    FROM serv_acct_item_0838_201308
    LIMIT 1
)
UNION
(
    SELECT
        'get_hour' AS format,
         CONVERT(SUBSTRING_INDEX(SUBSTRING_INDEX(START_TIME,' ',-1),':',1),UNSIGNED) AS time
    FROM serv_acct_item_0838_201308
    LIMIT 1
)
UNION
(
    SELECT
        'dayofweek' AS format,
         DAYOFWEEK(SUBSTRING_INDEX(START_TIME,' ',1)) AS time
    FROM serv_acct_item_0838_201308
    LIMIT 1
)

Running query in 'mysql+pyodbc://telecom'

4 rows affected.

,format,time
0,original,2013/8/1 0:00:12
1,get_date,2013/8/1
2,get_hour,0
3,dayofweek,5


```sql
CREATE TABLE sight_weekend_201308 (
	SELECT 
        LON, LAT, 
        COUNT(1) AS REC_NO
	FROM sight_xy_201308
	WHERE (WEEK_DAY=1 OR WEEK_DAY=7) AND (HOUR_NO>=10 AND HOUR_NO<22)
	GROUP BY LON, LAT
	ORDER BY COUNT(1) DESC
);
ALTER TABLE sight_weekend_201308 ADD RANK INT AUTO_INCREMENT KEY;
ALTER TABLE sight_weekend_201308 ADD CENTER_FLAG INT; 
UPDATE sight_weekend_201308 SET CENTER_FLAG=0;
SET @n=(
	SELECT COUNT(1) FROM sight_weekend_201308
);
UPDATE sight_weekend_201308 SET CENTER_FLAG=1
	WHERE RANK<=0.01*@n;
CREATE INDEX xy_weekend_201308 ON sight_weekend_201308(LON, LAT);
```

- The command `Count(1)` is euqivalent to `Count(*)` returning the number of 
observatons in a column.*
- week day = 1 is Sunday while 7 is Saturday
- the hot flag (in this case is the center flag) for a tower is to indicate the top 1% usage

```sql
CREATE TABLE tower_hot_201308 (
	SELECT 
        -- coluns from tower table
        A.CELL_10, A.CELL_16, A.LON, A.LAT, A.X, A.Y,
        -- create MAIN_CENTER_FLAG column
        -- 
        A.DEYANG_CENTER AS MAIN_CENTER_FLAG, 
		IFNULL(B.CENTER_FLAG,0) AS HOT_FLAG, 
        CONVERT(A.DEYANG_CENTER,UNSIGNED)*IFNULL(B.CENTER_FLAG,0) AS HOT_MAIN_FLAG, 
		IFNULL(B.CENTER_FLAG,0)-CONVERT(A.DEYANG_CENTER,UNSIGNED)*IFNULL(B.CENTER_FLAG,0) AS HOT_OTHER_FLAG
	FROM tower A LEFT JOIN sight_weekend_201308 B
	ON A.LON=B.LON AND A.LAT=B.LAT
);
CREATE INDEX cell_tower_201308 ON tower_hot_201308(CELL_16);
```

- IFNULL: returns a specified value if the expression is NULL.
- HOT_OTHER FLAG menas it's under heavy traffice but the position of the tower is not in the center of deyang center